In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [4]:
## Load dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
## removing feature that doesnot contain information
#Dropping irrelevant
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

KeyError: "['RowNumber', 'CustomerId', 'Surname'] not found in axis"

In [9]:
# Geography and gender are categorical features
# Lets encode the gender first
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])
data['Gender']

#converted female:0 and male:1

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [13]:
## Now Applying onehot encoding for Geography category.
## Label encoding is not remommended as there are many caterogries [0,1,2,3], NN might think label with 3 will be greater than 1
# One hot  encoding for Geography

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [19]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [15]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [20]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [21]:
# Combine one hot encoder columns with original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [22]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [24]:
## Split dataset to dependent and independent feature
X=data.drop('Exited', axis=1)
y=data['Exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

##Scale down this feature with standard scalar
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)



In [26]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [27]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Implementation

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [29]:
## BUILD ANN 
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #HL1 connected with input layer
    Dense(32, activation='relu'), #HL2
    Dense(1, activation='sigmoid') #output layer
])

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
#To customize the optimiser and Loss

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [33]:
## To do forward and backward propagation, model needs to compile
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [44]:
## setup the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callsback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [45]:
## Set early stopping - if model stops learning apply early stopping. If loss is not decreasing after certain epoch ---stop further training
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [46]:
## Train the model
history=model.fit(
    X_train, y_train,validation_data=(X_test,y_test), epochs=100,
    callbacks=[tensorflow_callsback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3418 - accuracy: 0.8591 - val_loss: 0.3414 - val_accuracy: 0.8605
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3399 - accuracy: 0.8608 - val_loss: 0.3445 - val_accuracy: 0.8570
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3351 - accuracy: 0.8643 - val_loss: 0.3348 - val_accuracy: 0.8610
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3331 - accuracy: 0.8637 - val_loss: 0.3379 - val_accuracy: 0.8550
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3260 - accuracy: 0.8674 - val_loss: 0.3533 - val_accuracy: 0.8545
Epoch 6/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3306 - accuracy: 0.8621 - val_loss: 0.3362 - val_accuracy: 0.8595
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3278 - accuracy: 0.8655 - val_loss: 0.3529 - val_accuracy: 0.8530

In [47]:
model.save('model.h5')

m:\Learning_Workspace\AI\AI_Projects\KrishNaikProjects\End-to-End-DL-ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
## Load Tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [51]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 20092), started 0:01:21 ago. (Use '!kill 20092' to kill it.)